In [1]:
!pip install transformers

In [2]:
!pip install datasets

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn import set_config; set_config(display='diagram')
from tensorflow.keras.preprocessing.text import Tokenizer
import string
import os
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, Sequential
from transformers import AutoTokenizer
from tensorflow.keras.callbacks import EarlyStopping
import datasets

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lucaspicot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#  Load the dataset already cleaned

In [4]:
data = pd.read_csv("/Users/lucaspicot/code/Zen1400/fake_news/raw_data/cleaned_df.csv")

In [5]:
## Delete the first column

data = data.drop(columns= 'Unnamed: 0')

In [6]:
data = data.dropna()

In [7]:
##Define the train test split

train_val_df = data.sample(frac = 0.8)
test_df = data.drop(train_val_df.index)

train_df = train_val_df.sample(frac = 0.8)
val_df = train_val_df.drop(train_df.index)

In [8]:
df_train = datasets.Dataset.from_pandas(train_df)

In [9]:
df_test = datasets.Dataset.from_pandas(test_df)

In [10]:
my_dataset_dict = datasets.DatasetDict({"train":df_train,"test":df_test})

# Preprocess

In [11]:
## tokenization

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

In [13]:
tokenized_data = my_dataset_dict.map(preprocess_function, batched = True)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/22 [00:00<?, ?ba/s]

In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

# Train

In [15]:
##train set and validation set

tf_train_set = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = tokenized_data["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

2022-09-12 19:31:55.484522: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
#parameters

from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5

es = EarlyStopping(patience = 5, verbose=2, monitor='val_loss', restore_best_weights = True)

batches_per_epoch = len(tokenized_data["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [17]:
#import pre-trained model distilbert base uncased

from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

2022-09-12 19:31:57.394492: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint 

In [18]:
#compile the model

import tensorflow as tf

model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics ='accuracy')

In [19]:
#freeze the weights of the first layers to only train the last layers


model.distilbert.trainable = False

In [ ]:
#train the model

model.fit(x=tf_train_set, validation_data=tf_validation_set, callbacks = [es], epochs=30, batch_size=batch_size)

2022-09-12 19:32:00.005684: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
   9/4236 [..............................] - ETA: 33:56:36 - loss: 1.1415 - accuracy: 0.4722

In [ ]:
model.save("/Users/lucaspicot/code/Zen1400/fake_news/notebooks/model_distilbert")